# Réalisez une veille technique : 
*Notebook Mission 2 - Réalisez une veille technique*

**Classification superviée d’images via les Visions Transformers (ViT)**

Il s'agit de réaliser un POC qui met en oeuvre cette nouvelle technique avec les données image déjà exploitées dans le cadre du projet 6.

In [27]:
# fonctions personnelles pour le pré traitement des données textuelles
import pre_treatment_text as ptt

import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

import os
import shutil

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
from torchmetrics.classification import Accuracy
from torch.optim.lr_scheduler import OneCycleLR

from transformers import ViTFeatureExtractor, ViTForImageClassification, AutoFeatureExtractor

from tqdm import tqdm

"""


from os import listdir

import torch
import timm
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Model
from transformers import ViTFeatureExtractor, ViTModel
from transformers import TFViTForImageClassification, ViTImageProcessor
from transformers import TFAutoModelForImageClassification, AutoFeatureExtractor
from tensorflow.keras.preprocessing import image_dataset_from_directory
from transformers import AdamW

#import shutil

"""
print("numpy version", np.__version__)
print("pandas version", pd.__version__)
print("matplotlib version", matplotlib.__version__)
#print("seaborn version", sns.__version__)
#print("scipy version", scipy.__version__)
#print("statsmodels version", statsmodels.__version__)

#print("sklearn version", sklearn.__version__)

pd.options.display.max_rows = 200
pd.options.display.max_columns = 100

numpy version 1.26.4
pandas version 2.1.4
matplotlib version 3.8.0


## 1 - Préparation des données

In [9]:
# Charger le fichier de données
data = pd.read_csv("C:/Users/admin/Documents/Projets/Projet_6/data_projet/Dataset+projet+prétraitement+textes+images/Flipkart/flipkart_com-ecommerce_sample_1050.csv", \
                   sep=',', low_memory=False)
data.head()

,uniq_id,crawl_timestamp,product_url,product_name,product_category_tree,pid,retail_price,discounted_price,image,is_FK_Advantage_product,description,product_rating,overall_rating,brand,product_specifications
0,55b85ea15a1536d46b7190ad6fff8ce7,2016-04-30 03:22:56 +0000,http://www.flipkart.com/elegance-polyester-mul...,Elegance Polyester Multicolor Abstract Eyelet ...,"[""Home Furnishing >> Curtains & Accessories >>...",CRNEG7BKMFFYHQ8Z,1899.0,899.0,55b85ea15a1536d46b7190ad6fff8ce7.jpg,False,Key Features of Elegance Polyester Multicolor ...,No rating available,No rating available,Elegance,"{""product_specification""=>[{""key""=>""Brand"", ""v..."
1,7b72c92c2f6c40268628ec5f14c6d590,2016-04-30 03:22:56 +0000,http://www.flipkart.com/sathiyas-cotton-bath-t...,Sathiyas Cotton Bath Towel,"[""Baby Care >> Baby Bath & Skin >> Baby Bath T...",BTWEGFZHGBXPHZUH,600.0,449.0,7b72c92c2f6c40268628ec5f14c6d590.jpg,False,Specifications of Sathiyas Cotton Bath Towel (...,No rating available,No rating available,Sathiyas,"{""product_specification""=>[{""key""=>""Machine Wa..."
2,64d5d4a258243731dc7bbb1eef49ad74,2016-04-30 03:22:56 +0000,http://www.flipkart.com/eurospa-cotton-terry-f...,Eurospa Cotton Terry Face Towel Set,"[""Baby Care >> Baby Bath & Skin >> Baby Bath T...",BTWEG6SHXTDB2A2Y,NaN,NaN,64d5d4a258243731dc7bbb1eef49ad74.jpg,False,Key Features of Eurospa Cotton Terry Face Towe...,No rating available,No rating available,Eurospa,"{""product_specification""=>[{""key""=>""Material"",..."
3,d4684dcdc759dd9cdf41504698d737d8,2016-06-20 08:49:52 +0000,http://www.flipkart.com/santosh-royal-fashion-...,SANTOSH ROYAL FASHION Cotton Printed King size...,"[""Home Furnishing >> Bed Linen >> Bedsheets >>...",BDSEJT9UQWHDUBH4,2699.0,1299.0,d4684dcdc759dd9cdf41504698d737d8.jpg,False,Key Features of SANTOSH ROYAL FASHION Cotton P...,No rating available,No rating available,SANTOSH ROYAL FASHION,"{""product_specification""=>[{""key""=>""Brand"", ""v..."
4,6325b6870c54cd47be6ebfbffa620ec7,2016-06-20 08:49:52 +0000,http://www.flipkart.com/jaipur-print-cotton-fl...,Jaipur Print Cotton Floral King sized Double B...,"[""Home Furnishing >> Bed Linen >> Bedsheets >>...",BDSEJTHNGWVGWWQU,2599.0,698.0,6325b6870c54cd47be6ebfbffa620ec7.jpg,False,Key Features of Jaipur Print Cotton Floral Kin...,No rating available,No rating available,Jaipur Print,"{""product_specification""=>[{""key""=>""Machine Wa..."


In [11]:
# Transformer l'arbre des catégories en 7 catégories principales
y = data['product_category_tree'].apply(lambda x: ptt.clean_category_first_level(x))
data['category'] = y

In [13]:
data.head()

,uniq_id,crawl_timestamp,product_url,product_name,product_category_tree,pid,retail_price,discounted_price,image,is_FK_Advantage_product,description,product_rating,overall_rating,brand,product_specifications,category
0,55b85ea15a1536d46b7190ad6fff8ce7,2016-04-30 03:22:56 +0000,http://www.flipkart.com/elegance-polyester-mul...,Elegance Polyester Multicolor Abstract Eyelet ...,"[""Home Furnishing >> Curtains & Accessories >>...",CRNEG7BKMFFYHQ8Z,1899.0,899.0,55b85ea15a1536d46b7190ad6fff8ce7.jpg,False,Key Features of Elegance Polyester Multicolor ...,No rating available,No rating available,Elegance,"{""product_specification""=>[{""key""=>""Brand"", ""v...",Home Furnishing
1,7b72c92c2f6c40268628ec5f14c6d590,2016-04-30 03:22:56 +0000,http://www.flipkart.com/sathiyas-cotton-bath-t...,Sathiyas Cotton Bath Towel,"[""Baby Care >> Baby Bath & Skin >> Baby Bath T...",BTWEGFZHGBXPHZUH,600.0,449.0,7b72c92c2f6c40268628ec5f14c6d590.jpg,False,Specifications of Sathiyas Cotton Bath Towel (...,No rating available,No rating available,Sathiyas,"{""product_specification""=>[{""key""=>""Machine Wa...",Baby Care
2,64d5d4a258243731dc7bbb1eef49ad74,2016-04-30 03:22:56 +0000,http://www.flipkart.com/eurospa-cotton-terry-f...,Eurospa Cotton Terry Face Towel Set,"[""Baby Care >> Baby Bath & Skin >> Baby Bath T...",BTWEG6SHXTDB2A2Y,NaN,NaN,64d5d4a258243731dc7bbb1eef49ad74.jpg,False,Key Features of Eurospa Cotton Terry Face Towe...,No rating available,No rating available,Eurospa,"{""product_specification""=>[{""key""=>""Material"",...",Baby Care
3,d4684dcdc759dd9cdf41504698d737d8,2016-06-20 08:49:52 +0000,http://www.flipkart.com/santosh-royal-fashion-...,SANTOSH ROYAL FASHION Cotton Printed King size...,"[""Home Furnishing >> Bed Linen >> Bedsheets >>...",BDSEJT9UQWHDUBH4,2699.0,1299.0,d4684dcdc759dd9cdf41504698d737d8.jpg,False,Key Features of SANTOSH ROYAL FASHION Cotton P...,No rating available,No rating available,SANTOSH ROYAL FASHION,"{""product_specification""=>[{""key""=>""Brand"", ""v...",Home Furnishing
4,6325b6870c54cd47be6ebfbffa620ec7,2016-06-20 08:49:52 +0000,http://www.flipkart.com/jaipur-print-cotton-fl...,Jaipur Print Cotton Floral King sized Double B...,"[""Home Furnishing >> Bed Linen >> Bedsheets >>...",BDSEJTHNGWVGWWQU,2599.0,698.0,6325b6870c54cd47be6ebfbffa620ec7.jpg,False,Key Features of Jaipur Print Cotton Floral Kin...,No rating available,No rating available,Jaipur Print,"{""product_specification""=>[{""key""=>""Machine Wa...",Home Furnishing


**Créons les dossiers train, val, test qui contiennent les images**

In [25]:
# Définir les dossiers contenant les images
source_folder = "C:/Users/admin/Documents/Projets/Projet_6/data_projet/Flipkart/Images/"  # Dossier où se trouvent les images
train_folder = "C:/Users/admin/Documents/Projets/Projet_6/data_projet/Flipkart/images_per_category"  
val_folder = "C:/Users/admin/Documents/Projets/Projet_6/data_projet/Flipkart/images_per_category_val"
test_folder = "C:/Users/admin/Documents/Projets/Projet_6/data_projet/Flipkart/images_per_category_test" 

# Séparer les données (80% train + validation, 30% test)
df_train_val, df_test = train_test_split(data, test_size=0.20, stratify=data['category'], random_state=42)

# Séparer les données (80% train + 20% validation)
df_train, df_val = train_test_split(df_train_val, test_size=0.20, stratify=df_train_val['category'], random_state=42)

def copy_images(df_subset, destination_folder):
    """ Fonction pour copier les images dans le dossier approprié. """
    for _, row in df_subset.iterrows():
        image_name = row['image']
        category = row['category']

        # Chemin source et destination
        src_path = os.path.join(source_folder, image_name)
        dest_folder = os.path.join(destination_folder, category)
        dest_path = os.path.join(dest_folder, image_name)

        # Créer le dossier s'il n'existe pas
        os.makedirs(dest_folder, exist_ok=True)

        # Vérifier si l'image existe avant de la copier
        if os.path.exists(src_path):
            shutil.copy2(src_path, dest_path)  # Copie sans supprimer l'original
            print(f"✅ {image_name} copié dans {dest_folder}")
        else:
            print(f"⚠️ Image non trouvée : {image_name}")

# Copier les images dans les dossiers appropriés
print("📂 Copie des images d'entraînement...")
copy_images(df_train, train_folder)

print("📂 Copie des images de validation...")
copy_images(df_val, val_folder)

print("📂 Copie des images de test...")
copy_images(df_test, test_folder)

print("✅ Opération terminée !")

📂 Copie des images d'entraînement...
✅ 762b6b22cccf91935e92446efe298fe3.jpg copié dans C:/Users/admin/Documents/Projets/Projet_6/data_projet/Flipkart/images_per_category\Baby Care
✅ fc93a41f31c4e997fbfcc5bd0a63ee65.jpg copié dans C:/Users/admin/Documents/Projets/Projet_6/data_projet/Flipkart/images_per_category\Watches
✅ 503bb08f7bdf6a395ec564dd1dc83619.jpg copié dans C:/Users/admin/Documents/Projets/Projet_6/data_projet/Flipkart/images_per_category\Kitchen  Dining
✅ e4922f01eda047582cd72e9d1063ab7a.jpg copié dans C:/Users/admin/Documents/Projets/Projet_6/data_projet/Flipkart/images_per_category\Home Decor  Festive Needs
✅ cc2acb156df3202f172b280132ab459a.jpg copié dans C:/Users/admin/Documents/Projets/Projet_6/data_projet/Flipkart/images_per_category\Home Furnishing
✅ d98147c96d2eebf723b2b67d791bcdaa.jpg copié dans C:/Users/admin/Documents/Projets/Projet_6/data_projet/Flipkart/images_per_category\Watches
✅ b91b13aceec0a140944a2c8915e45e59.jpg copié dans C:/Users/admin/Documents/Projet

**Créons les datasets "images" train, validation, et test**

In [30]:
# Define dataset paths
data_dir = "C:/Users/admin/Documents/Projets/Projet_6/data_projet/Flipkart"

# Define transformations for data preprocessing and augmentation
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

val_test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Create datasets and data loaders
train_dataset = datasets.ImageFolder(os.path.join(data_dir, "images_per_category"), transform=train_transform)
valid_dataset = datasets.ImageFolder(os.path.join(data_dir, "images_per_category_val"), transform=val_test_transform)
test_dataset = datasets.ImageFolder(os.path.join(data_dir, "images_per_category_test"), transform=val_test_transform)

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

## 2 - Classifier les images

Méthode non optimisée :

In [32]:
# Load the pre-trained ViT model
model_name = "google/vit-base-patch16-224-in21k"
feature_extractor = ViTFeatureExtractor(model_name)
model = ViTForImageClassification.from_pretrained(model_name, num_labels=len(train_dataset.classes))

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=0.0001)  # Adjust learning rate

# Learning rate scheduler with warm-up
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.001, total_steps=len(train_loader) * 20)

# Training loop
num_epochs = 20
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

best_validation_accuracy = 0.0

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs.logits, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    # Validation loop
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for inputs, labels in valid_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.logits, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        
        validation_accuracy = 100 * correct / total
        
        # Save the model if validation accuracy improved
        if validation_accuracy > best_validation_accuracy:
            best_validation_accuracy = validation_accuracy
            torch.save(model.state_dict(), "best_model.pth")
    
    print(f"Epoch [{epoch + 1}/{num_epochs}] "
          f"Loss: {running_loss / len(train_loader):.4f} "
          f"Validation Accuracy: {validation_accuracy:.2f}%")
    
    # Adjust learning rate
    scheduler.step()

# Load the best model for testing
model.load_state_dict(torch.load("best_model.pth"))

# Testing the model
model.eval()
test_correct = 0
test_total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.logits, 1)
        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()

test_accuracy = 100 * test_correct / test_total
print(f"Test Accuracy: {test_accuracy:.2f}%")

C:\Users\admin\anaconda3\Lib\site-packages\transformers\models\vit\feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(
Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\admin\anaconda3\Lib\site-packages\PIL\Image.py:3186: DecompressionBombWarning: Image size (93680328 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


Epoch [1/20] Loss: 1.8610 Validation Accuracy: 65.48%
Epoch [2/20] Loss: 1.6300 Validation Accuracy: 77.38%
Epoch [3/20] Loss: 1.3646 Validation Accuracy: 82.14%
Epoch [4/20] Loss: 1.1176 Validation Accuracy: 80.95%
Epoch [5/20] Loss: 0.9079 Validation Accuracy: 82.74%
Epoch [6/20] Loss: 0.7553 Validation Accuracy: 82.74%
Epoch [7/20] Loss: 0.6386 Validation Accuracy: 84.52%
Epoch [8/20] Loss: 0.5346 Validation Accuracy: 83.33%
Epoch [9/20] Loss: 0.4667 Validation Accuracy: 82.14%
Epoch [10/20] Loss: 0.4218 Validation Accuracy: 84.52%
Epoch [11/20] Loss: 0.3505 Validation Accuracy: 81.55%
Epoch [12/20] Loss: 0.3519 Validation Accuracy: 85.12%
Epoch [13/20] Loss: 0.2586 Validation Accuracy: 86.31%
Epoch [14/20] Loss: 0.2620 Validation Accuracy: 85.71%
Epoch [15/20] Loss: 0.2420 Validation Accuracy: 84.52%
Epoch [16/20] Loss: 0.2057 Validation Accuracy: 85.12%
Epoch [17/20] Loss: 0.2524 Validation Accuracy: 79.17%
Epoch [18/20] Loss: 0.2297 Validation Accuracy: 82.74%
Epoch [19/20] Loss:

**Méthode optimisée**

In [ ]:
# Load the pre-trained ViT model
model_name = "google/vit-base-patch16-224-in21k"
feature_extractor = AutoFeatureExtractor.from_pretrained(model_name)
model = ViTForImageClassification.from_pretrained(model_name, num_labels=len(train_dataset.classes))

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-4)

# Learning rate scheduler with warm-up
num_epochs = 20
total_steps = len(train_loader) * num_epochs
scheduler = OneCycleLR(optimizer, max_lr=1e-3, total_steps=total_steps)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Définir l'accuracy metric
train_accuracy_metric = Accuracy(task="multiclass", num_classes=len(train_dataset.classes)).to(device)
valid_accuracy_metric = Accuracy(task="multiclass", num_classes=len(train_dataset.classes)).to(device)
test_accuracy_metric = Accuracy(task="multiclass", num_classes=len(train_dataset.classes)).to(device)

# Training loop (avec torchmetrics)
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    train_accuracy_metric.reset()

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs.logits, labels)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()

        running_loss += loss.item()
        train_accuracy_metric.update(outputs.logits, labels)

    train_accuracy = train_accuracy_metric.compute() * 100

    # Validation loop
    model.eval()
    valid_accuracy_metric.reset()

    with torch.no_grad():
        for inputs, labels in valid_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            valid_accuracy_metric.update(outputs.logits, labels)

    validation_accuracy = valid_accuracy_metric.compute() * 100

    # Sauvegarde du meilleur modèle
    if validation_accuracy > best_validation_accuracy:
        best_validation_accuracy = validation_accuracy
        torch.save(model.state_dict(), "best_model.pth")

    print(f"Epoch [{epoch+1}/{num_epochs}] "
          f"Train Acc: {train_accuracy:.2f}% "
          f"Valid Acc: {validation_accuracy:.2f}% "
          f"Loss: {running_loss / len(train_loader):.4f}")

# Test loop avec torchmetrics
model.load_state_dict(torch.load("best_model.pth"))
model.eval()
test_accuracy_metric.reset()

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        test_accuracy_metric.update(outputs.logits, labels)

test_accuracy = test_accuracy_metric.compute() * 100
print(f"Test Accuracy: {test_accuracy:.2f}%")